In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
# ukr1 & ukr7 use the same code but different file path for validation
ukr_all = gpd.read_file("./ukr1/1_ukr1_all_footprint/ukr1_all_aoi.geojson")# all footprint
ukr_gee = gpd.read_file("./ukr1/2_ukr1_damaged_ttest/ukr1_final_20220224_12_4_vs_20220512_25.geojson") # gee results; here is an exaple of ukr1 with 2.5 threshold
ukr_unosat = gpd.read_file("./ukr1/3_ukr1_unosat_footprints/ukr1_unosat_damaged_footprint_aoi.geojson") # unosat damaged footprint; building level

In [3]:
# check data
print(ukr_all.shape)
print(ukr_gee.shape)
print(ukr_unosat.shape)

(19688, 10)
(9724, 11)
(4953, 10)


In [4]:
# check crs
print(ukr_all.crs)
print(ukr_gee.crs)
print(ukr_unosat.crs)

EPSG:3857
EPSG:4326
EPSG:3857


In [5]:
# transform crs
ukr_gee = ukr_gee.to_crs(ukr_unosat.crs)
print(ukr_gee.crs)

EPSG:3857


In [ ]:
# labels
ukr_all = ukr_all.copy()

# Ground Truth
ukr_all['gt_damaged'] = ukr_all.geometry.apply(
    lambda geom: ukr_unosat.intersects(geom).any()
)

# Pred_damaged
ukr_all['pred_damaged'] = ukr_all.geometry.apply(
    lambda geom: ukr_gee.intersects(geom).any()
)


In [ ]:
# define TP FP FN TN
def classify(row):
    if row['gt_damaged'] and row['pred_damaged']:
        return 'TP'
    elif not row['gt_damaged'] and row['pred_damaged']:
        return 'FP'
    elif row['gt_damaged'] and not row['pred_damaged']:
        return 'FN'
    else:
        return 'TN'

ukr_all['confusion'] = ukr_all.apply(classify, axis=1)

# summarize
confusion_counts = ukr_all['confusion'].value_counts()
TP = confusion_counts.get('TP', 0)
FP = confusion_counts.get('FP', 0)
FN = confusion_counts.get('FN', 0)
TN = confusion_counts.get('TN', 0)

In [ ]:
# Calculate
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
accuracy = (TP + TN) / (TP + FP + FN + TN)
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

In [ ]:
# Print results
print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"TN: {TN}\n")

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1_score:.4f}")